In [9]:
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 701 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [10]:
import pandas as pd
import string as st
import spacy
import time
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
import es_core_news_sm
sp = es_core_news_sm.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Let's us read the input data
df = pd.read_csv('/content/drive/MyDrive/TFM/data/export_licitaciones.csv', sep=',', encoding='UTF8')
df.head()

,id_cpv,id_division,id_grupo,id_clase,id_categoria,objeto
0,32500000,32000000,32500000.0,NaN,NaN,Adquisición de repuestos para el ILS AMS 2100
1,65310000,65000000,65300000.0,65310000.0,NaN,Suministro de energía eléctrica en el Centro d...
2,92112000,92000000,92100000.0,92110000.0,92112000.0,"Servicio de duplicado de copias de películas, ..."
3,9122110,9000000,9100000.0,9120000.0,9122000.0,Gas Licuado Propano 2013/2014
4,75000000,75000000,NaN,NaN,NaN,Servicio de vigilancia del Centro Cultural de ...


In [ ]:
df.isnull().sum()

id_cpv               0
id_division          0
id_grupo         42863
id_clase        104943
id_categoria    193597
objeto               0
dtype: int64

In [ ]:
df = df.sample(n=50000, random_state=1)
df

,id_capitulo,id_concepto,id_partida_generica,descripcion
246729,2000,2500,2530,Infliximab SOLUCIÓN INYECTABLE El frasco ámpul...
71686,3000,3200,3250,SERVICIO DE ARRENDAMIENTO INTEGRAL DE VEHÍCULO...
26815,6000,6100,6150,Invitación E350-2018 Seguimiento y control par...
294323,3000,3500,3510,TRABAJOS DE REHABILTACIÓN Y MANTTO. EN LA ESTA...
224244,2000,2500,2530,ADQUISICION DE INSUMOS DE LISOSOMALES (IDURSUL...
...,...,...,...,...
496893,3000,3300,3390,PRESTACIÓN DE SERVICIOS PROFESIONALES DGAQ
299737,3000,3200,3270,Suscripción anual del servicio de correo elect...
73851,3000,3500,3580,CONTRATACIÓN PLURIANUAL DEL SERVICIO INTEGRAL ...
157176,3000,3100,3170,CONTRATACION DE SERVICIO DE CONDUCCION DE SEÑA...


# Text cleaning and processing steps
* Remove punctuations
* Convert text to tokens
* Remove tokens of length less than or equal to 3
* Remove stopwords using NLTK corpus stopwords list to match
* Apply stemming
* Apply lemmatization
* Convert words to feature vectors

In [ ]:
def preprocessing_fastest(text):
  #tokenizamos words 
  token_word = nltk.word_tokenize(text.lower(), "spanish")
  index = 0
  while(index < len(token_word)):
    #omit tokens if are less than 4 characters or are punctuation marks
    if len(token_word[index]) <= 3 or token_word[index] in st.punctuation:
      token_word.pop(index)
    elif not (token_word[index].isalpha()):
      token_word.pop(index)
    #omit tokens if are stopwords in spanish
    elif token_word[index] in nltk.corpus.stopwords.words('spanish'):
      token_word.pop(index)
    else:
      index += 1
  #lematization of tokens
  doc = sp(' '.join(token_word))
  #for token in doc:
  #  print(token.text, token.lemma_, token.pos_, token.dep_)
  return " ".join([token.lemma_ for token in doc])

In [ ]:
def preprocessing_fastest_v2(text):
  #tokenizamos words 
  token_word = nltk.word_tokenize(text.lower(), "spanish")
  sentence = []
  for token in token_word:
    #omit tokens if are less than 4 characters or are punctuation marks
    if len(token) <= 3 or token in st.punctuation:
      continue
    #omit if the token has numbers
    if not (token.isalpha()):
      continue
    #omit tokens if are stopwords in spanish
    if token in nltk.corpus.stopwords.words('spanish'):
      continue
    sentence.append(token)
  #lematization of sentence
  doc = sp(' '.join(sentence))
  return ' '.join([token.lemma_ for token in doc])

In [ ]:
#Segunada version super mejorada: Más rápida
start = time.time()
print(preprocessing_fastest(df['objeto'].values[0]))
end = time.time()
print(end - start)

adquisición repuesto
0.05639839172363281


In [ ]:
#Tercera version super mejorada: La más rápida
start = time.time()
print(preprocessing_fastest_v2(df['objeto'].values[0]))
end = time.time()
print(end - start)

adquisición repuesto
0.07712173461914062


In [ ]:
#Versión la más rápida
start = time.time()
df['clean_text'] = df['objeto'].apply(lambda description: preprocessing_fastest_v2(description))
end = time.time()
print(end - start)

4703.552677869797


In [ ]:
df

,id_cpv,id_division,id_grupo,id_clase,id_categoria,objeto,clean_text
0,32500000,32000000,32500000.0,NaN,NaN,Adquisición de repuestos para el ILS AMS 2100,adquisición repuesto
1,65310000,65000000,65300000.0,65310000.0,NaN,Suministro de energía eléctrica en el Centro d...,suministrar energía eléctrico centrar conserva...
2,92112000,92000000,92100000.0,92110000.0,92112000.0,"Servicio de duplicado de copias de películas, ...",servicio duplicar copiar película tráiler vide...
3,9122110,9000000,9100000.0,9120000.0,9122000.0,Gas Licuado Propano 2013/2014,licuar propano
4,75000000,75000000,NaN,NaN,NaN,Servicio de vigilancia del Centro Cultural de ...,servicio vigilancia centrar cultural españa mé...
...,...,...,...,...,...,...,...
387390,70310000,70000000,70300000.0,70310000.0,NaN,"Arrendamiento cafetería, aseos y vestuarios de...",arrendamiento cafetería aseo vestuario edifici...
387391,90911000,90000000,90900000.0,90910000.0,90911000.0,Contrato basado en Acuerdo Marco servicio de l...,contratar basar acordar marcar servicio limpie...
387392,85100000,85000000,85100000.0,NaN,NaN,Servicio de prevención ajeno para el Instituto...,servicio prevención ajeno instituto municipal ...
387393,34631000,34000000,34600000.0,34630000.0,34631000.0,Certificacion conjunto pivote y tubo guia,certificacion conjuntar pivotar tubo guia


In [3]:
#Vemos aquellos registros que al preprocesar la descripción se vuelve nulo
df[df['clean_text'].isnull()]

,id_cpv,id_division,id_grupo,id_clase,id_categoria,objeto,clean_text
2376,15000000,15000000,NaN,NaN,NaN,Pan 2013,NaN
9865,50100000,50000000,50100000.0,NaN,NaN,"Exp. N º 2 6 Bae,s Madrid-Toledo",NaN
10392,50100000,50000000,50100000.0,NaN,NaN,"Exp. 4 7 Bae,s Centro-Sur",NaN
10786,15000000,15000000,NaN,NaN,NaN,Pan ta2014,NaN
17296,38000000,38000000,NaN,NaN,NaN,Gps's,NaN
...,...,...,...,...,...,...,...
381353,60130000,60000000,60100000.0,60130000.0,NaN,gu.transporte escolar.ruta 51. Mayo-Junio 2022,NaN
381376,60130000,60000000,60100000.0,60130000.0,NaN,gu.transporte escolar.ruta 80. Mayo-Junio 2022,NaN
382407,34631000,34000000,34600000.0,34630000.0,34631000.0,.suministro de gc 5210 caucho-metal 2022-04,NaN
382589,60130000,60000000,60100000.0,60130000.0,NaN,gu.transporte escolar.ruta 163. Mayo-Junio 2022,NaN


In [5]:
#Vemos la cantidad de registros iniciales
df.groupby(['id_division']).count()

,id_cpv,id_grupo,id_clase,id_categoria,objeto,clean_text
id_division,,,,,,
3000000,3209,2105,1619,896,3209,3209
9000000,7625,7407,6735,3205,7625,7625
14000000,1011,977,871,449,1011,1009
15000000,4087,3008,1898,1324,4087,4085
16000000,572,472,180,22,572,572
18000000,3764,3456,2238,1644,3764,3762
19000000,527,501,359,60,527,526
22000000,2713,2401,1319,930,2713,2710
24000000,2345,2116,1571,1213,2345,2344


In [4]:
#Vemos aquellos registros que al preprocesar la descripción se vuelve nulo
df[df['clean_text'].isnull()].groupby(['id_division']).count()

,id_cpv,id_grupo,id_clase,id_categoria,objeto,clean_text
id_division,,,,,,
14000000,2,1,0,0,2,0
15000000,2,0,0,0,2,0
18000000,2,2,2,0,2,0
19000000,1,1,0,0,1,0
22000000,3,2,2,0,3,0
24000000,1,1,1,1,1,0
30000000,3,3,3,3,3,0
31000000,7,5,5,4,7,0
32000000,1,1,0,0,1,0


In [6]:
#Filtramos los nulos
df = df[df['clean_text'].notnull()]

In [7]:
#Vemos si han sido eliminados
df[df['clean_text'].isnull()]

,id_cpv,id_division,id_grupo,id_clase,id_categoria,objeto,clean_text


In [8]:
#Exportamos los registros limpiados a un nuevo dataset. 
df.to_csv('export_licitaciones_cleaned.csv', sep=',', encoding='UTF8', index=False)